# Configuração Inicial - Spark, DeltaLake e MinIO

Este notebook configura o ambiente PySpark com DeltaLake e conexão ao MinIO para armazenamento de dados.

In [ ]:
# Instalação de dependências (executar apenas uma vez)
# !pip install pyspark delta-spark minio boto3

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta import configure_spark_with_delta_pip
import os

# Configurações do MinIO
MINIO_ENDPOINT = os.getenv('MINIO_ENDPOINT', 'localhost:9000')
MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY', 'minioadmin')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY', 'minioadmin')
MINIO_BUCKET = os.getenv('MINIO_BUCKET', 'enderecos')

# Configuração do Spark com DeltaLake
builder = SparkSession.builder \
    .appName("MotorCorrespondenciaEnderecos") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:2.4.0,org.apache.hadoop:hadoop-aws:3.3.2") \
    .config("spark.hadoop.fs.s3a.endpoint", f"http://{MINIO_ENDPOINT}") \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .master("local[*]")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Configurar log level
spark.sparkContext.setLogLevel("WARN")

print("Spark Session criada com sucesso!")
print(f"Spark Version: {spark.version}")
print(f"MinIO Endpoint: {MINIO_ENDPOINT}")
print(f"Bucket: {MINIO_BUCKET}")

In [ ]:
# Definir caminhos base no MinIO
BASE_PATH = f"s3a://{MINIO_BUCKET}"

# Caminhos para cada camada
PATH_BRONZE = f"{BASE_PATH}/bronze"  # Dados brutos
PATH_SILVER = f"{BASE_PATH}/silver"  # Dados normalizados (Camada Prata)
PATH_GOLD = f"{BASE_PATH}/gold"       # Dados canônicos (Camada Ouro)

# Caminhos específicos
PATH_ENDERECOS_LIVRES = f"{PATH_BRONZE}/enderecos_livres"
PATH_DNE = f"{PATH_BRONZE}/dne"
PATH_CNEFE = f"{PATH_BRONZE}/cnefe"
PATH_OSM = f"{PATH_BRONZE}/osm"

PATH_ENDERECOS_ESTRUTURADOS = f"{PATH_SILVER}/enderecos_estruturados"
PATH_ENDERECOS_NORMALIZADOS = f"{PATH_SILVER}/enderecos_normalizados"

PATH_CAMADA_OURO = f"{PATH_GOLD}/camada_ouro"
PATH_CLUSTERS = f"{PATH_GOLD}/clusters"
PATH_MATCHES = f"{PATH_GOLD}/matches"

print("Caminhos configurados:")
print(f"Bronze: {PATH_BRONZE}")
print(f"Silver: {PATH_SILVER}")
print(f"Gold: {PATH_GOLD}")

In [ ]:
# Função auxiliar para salvar DataFrame como Delta Table
def save_delta_table(df, path, mode="overwrite", partition_by=None):
    """
    Salva um DataFrame como tabela Delta no MinIO
    
    Args:
        df: DataFrame do Spark
        path: Caminho no MinIO (s3a://)
        mode: Modo de escrita (overwrite, append, errorIfExists)
        partition_by: Lista de colunas para particionamento
    """
    writer = df.write.format("delta").mode(mode)
    
    if partition_by:
        writer = writer.partitionBy(*partition_by)
    
    writer.save(path)
    print(f"Dados salvos em: {path}")

# Função auxiliar para ler Delta Table
def read_delta_table(path):
    """
    Lê uma tabela Delta do MinIO
    
    Args:
        path: Caminho no MinIO (s3a://)
    
    Returns:
        DataFrame do Spark
    """
    return spark.read.format("delta").load(path)

print("Funções auxiliares definidas!")